In [ ]:
# Train model
import sys
sys.path.append(r'D:\skin_disease_detection\backend\app')
# from models import pretrained_model

In [ ]:
# Import saved model
from keras.models import load_model
model = load_model(r'D:\skin_disease_detection\backend\app\static\trained_models\skin_disease_model.h5')
print(model)

In [1]:
import sys
import pandas as pd
sys.path.append(r'D:\skin_disease_detection\backend\app')
from utils.preprocess import get_data_generators
from ai_models.ensemble_model import run_ensemble_model

# Prepare data generators
train_generator, validation_generator, test_generator, label_encoder, class_weights = get_data_generators()

# Run the ensemble model
run_ensemble_model(train_generator, validation_generator, test_generator, class_weights)

########## GETTING DATA GENERATORS ##########

Loading metadata...


########## TOP FIVE SAMPLE DATA ##########
Metadata:      lesion_id      image_id   dx dx_type   age   sex localization  \
0  HAM_0000118  ISIC_0027419  bkl   histo  80.0  male        scalp   
1  HAM_0000118  ISIC_0025030  bkl   histo  80.0  male        scalp   
2  HAM_0002730  ISIC_0026769  bkl   histo  80.0  male        scalp   
3  HAM_0002730  ISIC_0025661  bkl   histo  80.0  male        scalp   
4  HAM_0001466  ISIC_0031633  bkl   histo  75.0  male          ear   

                                                path  
0  D:\skin_disease_detection\backend\data\Ham1000...  
1  D:\skin_disease_detection\backend\data\Ham1000...  
2  D:\skin_disease_detection\backend\data\Ham1000...  
3  D:\skin_disease_detection\backend\data\Ham1000...  
4  D:\skin_disease_detection\backend\data\Ham1000...  


########## LABELS ##########
Encoding labels...
Labels encoded. Unique classes: ['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc']


c:\Users\Purve\anaconda3\envs\skn-dis-env-2\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 434s 2s/step - accuracy: 0.6388 - loss: 1.3524 - val_accuracy: 0.6542 - val_loss: 1.1634
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 369s 2s/step - accuracy: 0.6800 - loss: 1.1498 - val_accuracy: 0.6542 - val_loss: 1.1806
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 378s 2s/step - accuracy: 0.6718 - loss: 1.1633 - val_accuracy: 0.6542 - val_loss: 1.1703
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 376s 2s/step - accuracy: 0.6779 - loss: 1.1330 - val_accuracy: 0.6542 - val_loss: 1.1737


SAVED ResNet50 model to ../../trained_models\ResNet50_model.h5
64/64 ━━━━━━━━━━━━━━━━━━━━ 93s 1s/step


########## TRAINING : EfficientNetB0 ##########
Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 219s 1s/step - accuracy: 0.6488 - loss: 1.2503 - val_accuracy: 0.6542 - val_loss: 1.1834
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 207s 1s/step - accuracy: 0.6814 - loss: 1.1422 - val_accuracy: 0.6542 - val_loss: 1.1699
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 194s 1s/step - accuracy: 0.6747 - loss: 1.1579 - val_accuracy: 0.6542 - val_loss: 1.1683
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - accuracy: 0.6787 - loss: 1.1410 - val_accuracy: 0.6542 - val_loss: 1.1724
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 208s 1s/step - accuracy: 0.6777 - loss: 1.1508 - val_accuracy: 0.6542 - val_loss: 1.1656
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 206s 1s/step - accuracy: 0.6708 - loss: 1.1639 - val_accuracy: 0.6542 - val_loss: 1.1655
Epoch 7/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - accuracy: 0.6888 - 

SAVED EfficientNetB0 model to ../../trained_models\EfficientNetB0_model.h5
64/64 ━━━━━━━━━━━━━━━━━━━━ 41s 612ms/step


########## TRAINING : DenseNet121 ##########
Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 432s 2s/step - accuracy: 0.6348 - loss: 1.1987 - val_accuracy: 0.6858 - val_loss: 0.8334
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 409s 2s/step - accuracy: 0.6956 - loss: 0.8303 - val_accuracy: 0.6991 - val_loss: 0.8081
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 410s 2s/step - accuracy: 0.7283 - loss: 0.7729 - val_accuracy: 0.7238 - val_loss: 0.7558
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 379s 2s/step - accuracy: 0.7304 - loss: 0.7424 - val_accuracy: 0.7273 - val_loss: 0.7619
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 395s 2s/step - accuracy: 0.7331 - loss: 0.7220 - val_accuracy: 0.7243 - val_loss: 0.7693
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 393s 2s/step - accuracy: 0.7343 - loss: 0.7198 - val_accuracy: 0.7302 - val_loss: 0.7572


SAVED DenseNet121 model to ../../trained_models\DenseNet121_model.h5
64/64 ━━━━━━━━━━━━━━━━━━━━ 103s 2s/step


########## STACKING PREDICTIONS FROM BASE MODELS ##########
Shape of base_model_preds: (2024, 21)
Length of validation_generator.classes: 2024


########## TRAINING : Meta Model Logistic Regression ##########
SAVED meta-model to ../../trained_models\meta_model.pkl


########## EVALUATING ##########


c:\Users\Purve\anaconda3\envs\skn-dis-env-2\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 38s 599ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step


########## CLASSIFICATION REPORT ##########
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        66
           1       0.00      0.00      0.00       105
           2       0.00      0.00      0.00       241
           3       0.00      0.00      0.00        35
           4       0.00      0.00      0.00       212
           5       0.66      1.00      0.79      1319
           6       0.00      0.00      0.00        22

    accuracy                           0.66      2000
   macro avg       0.09      0.14      0.11      2000
weighted avg       0.43      0.66      0.52      2000



########## CONFUSION MATRIX ##########
[[   0    0    0    0    0   66    0]
 [   0    0    0    0    0  105    0]
 [   0    0    0    0    0  241    0]
 [   0    0    0    0    0   35    0]
 [   0    0    0    0    0  212    0]
 [   0

c:\Users\Purve\anaconda3\envs\skn-dis-env-2\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Purve\anaconda3\envs\skn-dis-env-2\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Purve\anaconda3\envs\skn-dis-env-2\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i